In [8]:
pwd

'/mmfs1/data/aglinska/BC-MRI-ADHD/Code'

In [9]:
from helper_funcs import *
from make_models import *

In [10]:
import os # Esoteric GPU stuff
os.environ['CUDA_VISIBLE_DEVICES']='2'

In [11]:
from tqdm import tqdm
from matplotlib import pyplot as plt

In [12]:
data_loc = '../Data/ABIDE-Anat-64iso-S982.npz'

In [17]:
class vae_data_loader():
    ''' this is the dataloader for VAE'''
    
    def __init__(self,data_dir,batch_size=32): # Init runs when you initialize the class

        self.data = np.load(data_loc)['data']
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.n_batches = int(self.data.shape[0]/self.batch_size)-1
        self.e = -1
        self.b = -1
        
        self.new_epoch()
        
    def new_epoch(self):
        self.e+=1
        self.perm_idx = np.random.permutation(np.arange(self.data.shape[0]))
        self.data = self.data[self.perm_idx,:,:,:]
        
        
    def get_batch(self):
        
        if self.b==self.n_batches:
            self.new_epoch()
            self.b = -1
                    
        self.b+=1        
        self.batch_idx = np.arange(self.b*self.batch_size,self.b*self.batch_size+self.batch_size)
        self.batch = self.data[self.batch_idx,:,:,:]
                    
        return self.batch

In [18]:
loader = vae_data_loader(data_dir=data_loc,batch_size=32)

In [21]:
loader.data.shape

(982, 64, 64, 64)

In [92]:
loader.get_batch();
print(f'{loader.e} / {loader.b}')

2 / 3


In [19]:
# VAE MODEL DEFINITION
encoder, decoder, vae = get_MRI_VAE_3D(input_shape=(64,64,64,1),
                                       latent_dim=2,
                                       batch_size = 32,
                                       disentangle=False,
                                       gamma=1,
                                       kernel_size = 3,
                                       filters = 16,
                                       intermediate_dim = 128,
                                       opt=None)

In [93]:
loss = []
for epoch in tqdm(range(2)):
    for batch in range(loader.n_batches):
        data_batch = loader.get_batch()
        batch_loss = vae.train_on_batch(data_batch)
        loss.append(loss)

100%|██████████| 2/2 [00:16<00:00,  8.20s/it]


In [ ]:
plt.plot(loss)

In [ ]:
loss

In [ ]:
#vae.predict(data_batch)